# Get jsons from fine-tuning experiments and analyze

## 0816-dataene er de som skal bli til modell som lastes opp til HuggingFace.
Treningen ble gjort på Fox. Vi zippet de to mappene og overførte direkte, uten github.


In [19]:
import json, os, sys, io
from pathlib import Path
import pandas as pd
from datetime import date
import zipfile
zip_config = zipfile.Path("tsa-0823/0823-configs.zip", at="configs/fox/")

In [4]:
# scp -r ec-egilron@fox.educloud.no:/fp/homes01/u01/ec-egilron/sqlabel-github/0816_all-logs.zip tsa-0816


'scp -r ec-egilron@fox.educloud.no:/fp/homes01/u01/ec-egilron/sqlabel-github/0816_all-logs.zip tsa-0816'

In [29]:
name_filter = "0823"
jsons = [f for f in zip_config.iterdir() if f.name.startswith(name_filter) and  f.name.endswith("json")]
records = []
name_filter = "0823"
root_keys = ["timestamp",  "task",  "machinery",  "best_epoch" ]
args_keys = [ 'model_name_or_path', 'task_name', 'dataset_name', 'output_dir', 'seed' , 'per_device_train_batch_size',  'learning_rate' , 'num_train_epochs']
epoch_keys = ['eval_loss', 'eval_precision', 'eval_recall', 'eval_f1', 'eval_accuracy','epoch', 'step' ]
for j in jsons:
    jdata = json.loads(j.read_text())
    for epoch_data in jdata["train_epochs_val"]:
        record = {k:v for k,v in jdata.items() if k in root_keys}
        record["config-file"] = j.name
        record.update({k:v for k,v in jdata["args_dict"].items() if k in args_keys})
        record.update({k:v for k,v in epoch_data.items() if k in epoch_keys})
        records.append(record)
df = pd.DataFrame.from_records(records)
print("Shape:", df.shape)
df.to_excel("tsa-0823/0823-finetuning_tsa.xlsx")

Shape: (192, 20)


In [31]:
wanteds = [ "ltg/norbert3-large",
           "ltg/norbert3-base"]
for wanted in wanteds:
    for task, group in df.groupby("task"):
        print(task, wanted)
        print(group[["config-file","model_name_or_path", "epoch", 'step' ,  'seed', "eval_f1", ]][group.seed == 303]
            .query("model_name_or_path in @wanted")
            .sort_values("eval_f1", ascending=False)
            .head(3))

tsa-bin ltg/norbert3-large
                                   config-file  model_name_or_path  epoch  \
144  08231013_tsa-bin_NorBERT_3_large_303.json  ltg/norbert3-large    5.0   
145  08231013_tsa-bin_NorBERT_3_large_303.json  ltg/norbert3-large   13.0   
146  08231013_tsa-bin_NorBERT_3_large_303.json  ltg/norbert3-large   14.0   

     step  seed   eval_f1  
144  1350   303  0.564346  
145  3510   303  0.563510  
146  3780   303  0.561877  
tsa-intensity ltg/norbert3-large
                                        config-file  model_name_or_path  \
32  08231013_tsa-intensity_NorBERT_3_large_303.json  ltg/norbert3-large   
33  08231013_tsa-intensity_NorBERT_3_large_303.json  ltg/norbert3-large   
34  08231013_tsa-intensity_NorBERT_3_large_303.json  ltg/norbert3-large   

    epoch  step  seed   eval_f1  
32   13.0  3510   303  0.325503  
33    8.0  2160   303  0.325146  
34    5.0  1350   303  0.319615  
tsa-bin ltg/norbert3-base
                                  config-file model_name

Further analysis in [0816-finetuning_tsa.xlsx](https://uio-my.sharepoint.com/:x:/r/personal/egilron_uio_no/Documents/SANT/TSA-HF/0816-finetuning_tsa.xlsx?d=wd099d0b4cb034234ab14f0b5daa8e13a&csf=1&web=1&e=mUCjZm)

In [28]:
jdata

{'timestamp': '08231013',
 'task': 'tsa-bin',
 'model_shortname': 'NorBERT_3_base',
 'machinery': 'fox',
 'args_dict': {'model_name_or_path': 'ltg/norbert3-base',
  'dataset_name': 'data/tsa_binary',
  'seed': 202,
  'per_device_train_batch_size': 64,
  'task_name': '08161531_tsa-bin_NorBERT_3_base',
  'output_dir': '/cluster/work/projects/ec30/egilron/08231013_tsa-bin_NorBERT_3_base_202',
  'overwrite_cache': True,
  'overwrite_output_dir': True,
  'do_train': True,
  'num_train_epochs': 16,
  'do_eval': True,
  'return_entity_level_metrics': False,
  'use_auth_token': False,
  'logging_strategy': 'epoch',
  'save_strategy': 'epoch',
  'evaluation_strategy': 'epoch',
  'save_total_limit': 2,
  'load_best_model_at_end': True,
  'label_column_name': 'tsa_tags',
  'disable_tqdm': True,
  'do_predict': True,
  'text_column_name': 'tokens',
  'learning_rate': 1e-05},
 'best_epoch': 9,
 'train_epochs_val': [{'eval_loss': 0.20297932624816895,
   'eval_precision': 0.5087527352297593,
   'eval